In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.compose import TransformedTargetRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import r2_score, mean_squared_error
import matplotlib.pyplot as plt

In [2]:
DATA_PATH = "data/data.csv"

df = pd.read_csv(DATA_PATH)
df.head()


,date,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,condition,sqft_above,sqft_basement,yr_built,yr_renovated,street,city,statezip,country,price
0,2014-05-02 00:00:00,3.0,1.50,1340,7912,1.5,0,0,3.0,1340,0,1955.0,2005,18810 Densmore Ave N,Shoreline,WA 98133,USA,313000.0
1,2014-05-02 00:00:00,5.0,2.50,3650,9050,2.0,0,4,5.0,3370,280,1921.0,0,709 W Blaine St,Seattle,WA 98119,USA,2384000.0
2,2014-05-02 00:00:00,3.0,2.00,1930,11947,1.0,0,0,4.0,1930,0,1966.0,0,26206-26214 143rd Ave SE,Kent,WA 98042,USA,342000.0
3,2014-05-02 00:00:00,3.0,2.25,2000,8030,1.0,0,0,4.0,1000,1000,1963.0,0,857 170th Pl NE,Bellevue,WA 98008,USA,420000.0
4,2014-05-02 00:00:00,4.0,2.50,1940,10500,1.0,0,0,4.0,1140,800,1976.0,1992,9105 170th Ave NE,Redmond,WA 98052,USA,550000.0


In [3]:
# Выводим структурные сведенья о DataFrame
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4600 entries, 0 to 4599
Data columns (total 18 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   date           4600 non-null   object 
 1   bedrooms       4600 non-null   float64
 2   bathrooms      4600 non-null   float64
 3   sqft_living    4600 non-null   int64  
 4   sqft_lot       4600 non-null   int64  
 5   floors         4600 non-null   float64
 6   waterfront     4600 non-null   int64  
 7   view           4600 non-null   int64  
 8   condition      4520 non-null   float64
 9   sqft_above     4600 non-null   int64  
 10  sqft_basement  4600 non-null   int64  
 11  yr_built       4390 non-null   float64
 12  yr_renovated   4600 non-null   int64  
 13  street         4600 non-null   object 
 14  city           4589 non-null   object 
 15  statezip       4600 non-null   object 
 16  country        4600 non-null   object 
 17  price          4600 non-null   float64
dtypes: float

Структура следующая:
date – дата продажи
bedrooms – количество спален
bathrooms – количество ванных комнат
sqft_living – жилая площадь
sqft_lot – общая площадь участка
floors – количество этажей
waterfront – наличие выхода к воде
view – оценка вида
condition – оценка состояния дома
sqft_above – площадь над уровнем земли
sqft_basement – площадь под уровнем земли
yr_built – год постройки дома
yr_renovated – год последнего ремонта
street – название улицы
city – город
statezip – комбинация штата и ZIP-кода
country – страна

In [ ]:
# Выводит статистику по столбцам
df.describe(include='all')


Числовые признаки:
count - количество не пустых значений,
mean - среднее арифметическое,
std - стандартное отклонение,
min, 25%, 50%, 75%, max - значения.

Категориальные/строковые признаки:
unique - количество уникальных элементов,
top - самое частое значение,
freq - частота самого частого значения.

Заметки по данным:
date – строковый признак, требует преобразования в datetime (преобразуем в строку, можно будет использовать вместе с yr_built, месяц возможно полезен, но не точно)
bedrooms – числовой признак, распределён нормально (оставляем как есть)
bathrooms – числовой признак, распределён нормально (оставляем как есть)
sqft_living – числовой признак, скошено вправо (применяем log)
sqft_lot – числовой признак, сильно скошено вправо (применяем log)
floors – дискретный числовой признак, нормальная форма (оставляем как есть)
waterfront – бинарный признак 0/1 (оставляем как есть)
view – дискретный рейтинговый признак 0–4 (оставляем как есть)
condition – дискретный рейтинговый признак 1–5 (оставляем как есть)
sqft_above – числовой признак, скошено вправо (применяем log)
sqft_basement – числовой признак, скошено вправо (применяем log)
yr_built – числовой признак, монотонный диапазон (преобразуем в возраст)
yr_renovated – числовой признак, почти всегда 0 (преобразуем в бинарный ремонт)
street – категориальный признак высокой кардинальности, значения почти уникальны (не используем)
city – категориальный признак высокой кардинальности (не используем)
statezip – категориальный признак высокой кардинальности (не используем)
country – константа, одно значение (не используем)

In [ ]:
# Показывает количество пропусков в каждом столбце
df.isna().sum()

condition - 80 пропусков (можно заполнить медианой)
yr_built - 210 пропусков (можно заполнить медианой)
city - 11 пропусков (уже не используем)


In [ ]:
# ПРЕДОБРАБОТКА

# Преобразовываем дату
df['date'] = pd.to_datetime(df['date'])

df['sale_year'] = df['date'].dt.year
df['sale_month'] = df['date'].dt.month

df = df.drop(columns=['date'])

# Обрабатываем пропуски
df['condition'] = df['condition'].fillna(df['condition'].median())
df['yr_built'] = df['yr_built'].fillna(df['yr_built'].median())
df['city'] = df['city'].fillna('unknown')

# Более удобные признаки
df['house_age'] = df['sale_year'] - df['yr_built']
df = df.drop(columns=['sale_year'])
df['renovated'] = (df['yr_renovated'] > 0).astype(int)
df = df.drop(columns=['yr_built', 'yr_renovated'])

# Удаляем не используемые признаки
df = df.drop(columns=['street', 'statezip','city','country'])

# Логарифмируем (log1p = log(x+1), безопасная операция с нулями)
for col in ['sqft_living', 'sqft_lot', 'sqft_above', 'sqft_basement']:
    df[col] = np.log1p(df[col])


In [ ]:
df.info()


In [ ]:
df.describe()


In [ ]:
y = df['price']
X = df.drop(columns=['price'])

In [ ]:
# Разделяем на train/test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

In [ ]:
# Обучение модели
model = LinearRegression()
model.fit(X_train, y_train)

pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import r2_score, mean_squared_error


print("R2:", r2_score(y_test, pred))
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)



Пока всё грустно и печально))

In [ ]:
# работаем с уже предобработанным df (до разбиения)
corr = df.corr(numeric_only=True)['price'].sort_values(ascending=False)
corr


In [ ]:
df['sqft_living_sq'] = df['sqft_living'] ** 2
df['bath_bed_ratio'] = df['bathrooms'] / df['bedrooms']
df = df.dropna(subset=['bath_bed_ratio'])
df_orign = df.copy()

In [ ]:
y = df['price']
X = df.drop(columns=['price'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)
model = LinearRegression()
model.fit(X_train, y_train)

pred = model.predict(X_test)

print("R2:", r2_score(y_test, pred))
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)


In [ ]:
Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.3 * IQR
upper_bound = Q3 + 1.3 * IQR
df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]

y = df['price']
X = df.drop(columns=['price'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=pd.cut(y, bins=5)
)
model = LinearRegression()
model.fit(X_train, y_train)

pred = model.predict(X_test)

print("R2:", r2_score(y_test, pred))
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)


In [ ]:
model = Ridge(alpha=1, random_state=42)
model.fit(X_train, y_train)

pred = model.predict(X_test)

print("R2:", r2_score(y_test, pred))
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

In [ ]:
model = Lasso(alpha=1, random_state=42)
model.fit(X_train, y_train)

pred = model.predict(X_test)

print("R2:", r2_score(y_test, pred))
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

In [ ]:
model = ElasticNet(alpha=0.05, l1_ratio=0.05, random_state=42)
model.fit(X_train, y_train)

pred = model.predict(X_test)

print("R2:", r2_score(y_test, pred))
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)

Продолжаем

In [ ]:
df = df_orign.copy()

Q1 = df['price'].quantile(0.25)
Q3 = df['price'].quantile(0.75)
IQR = Q3 - Q1
lower_bound = Q1 - 1.3 * IQR
upper_bound = Q3 + 1.3 * IQR
df = df[(df['price'] >= lower_bound) & (df['price'] <= upper_bound)]

y = df['price']
X = df.drop(columns=['price'])

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=pd.cut(y, bins=5)
)
model = LinearRegression()
model.fit(X_train, y_train)

pred = model.predict(X_test)

print("R2:", r2_score(y_test, pred))
mse = mean_squared_error(y_test, pred)
rmse = np.sqrt(mse)
print("RMSE:", rmse)